# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [42]:
# Libraries
import pandas as pd
import sqlalchemy
import numpy as np
import re

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [43]:
# your code here
df = pd.read_csv("Admission_Predict.csv")

Remove trailing spaces at the end of the column names if there are any.


In [44]:
# your code here
df.columns = [x.strip() for x in df.columns]

Let's evaluate the dataset by looking at the `head` function.

In [45]:
# your code here
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [46]:
# your code here
df.isnull().any()

Serial No.           False
GRE Score            False
TOEFL Score          False
University Rating    False
SOP                  False
LOR                  False
CGPA                 False
Research             False
Chance of Admit      False
dtype: bool

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [47]:
# your code here
df.index = df['Serial No.']
df.index=df.index - 1
df
#df.set_index('Serial No.')

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,,
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [48]:
# your code here
print(df['GRE Score'].duplicated().any())
print(df['CGPA'].duplicated().any())

True
True


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [49]:
# your code here
#Set the index value start from zero 
#df.index=df.index - 1
#df.reset_index(drop=True)


In [ ]:
#df.set_index(['GRE Score','CGPA'],inplace=True) # Multiple indexing
#df

# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [50]:
# your code here
df[(df.CGPA > 9 )& (df.Research == 1)].count()

Serial No.           104
GRE Score            104
TOEFL Score          104
University Rating    104
SOP                  104
LOR                  104
CGPA                 104
Research             104
Chance of Admit      104
dtype: int64

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [51]:
# your code here
df[(df.CGPA > 9 )& (df.SOP < 3.5)].mean()['Chance of Admit']

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [52]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here
    return (col - col.mean())/col.std(ddof=0)

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [53]:
# your code here
df.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

In [54]:
df['CGPA_std'] = standardize(df['CGPA'])
df['GRE_std'] = standardize(df['GRE Score'])
df['LOR_std'] = standardize(df['LOR'])

We will generate the decision choice at random using the code below. Please run the cell.

In [55]:
# Libraries
from random import choices

In [56]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=df.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [57]:
# your code here
df['deciding_column'] = df.lookup(df.index, decision_choice)

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [58]:
# your code here
df['decision'] = np.where(df['deciding_column'] > 0.8, 1, 0)

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [59]:
# your code here
df['decision'].value_counts()

0    307
1     93
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [60]:
# your code here
df.columns = [re.sub('\W', '', x.replace(' ','_').lower()) for x in df.columns]

In [61]:
df.columns

Index(['serial_no', 'gre_score', 'toefl_score', 'university_rating', 'sop',
       'lor', 'cgpa', 'research', 'chance_of_admit', 'cgpa_std', 'gre_std',
       'lor_std', 'deciding_column', 'decision'],
      dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [62]:
# your code here
df['gre_score'] = df[['university_rating', 'gre_score']][df['university_rating'] > 4]['gre_score'] + 10

In [63]:
df['adjusted_gre'] = pd.cut(df['gre_score'], 4)